In [1]:
from streamlit.connections import ExperimentalBaseConnection
import os
import requests
import streamlit as st
import json
import plotly.graph_objects as go


In [2]:
session = requests.Session()

In [3]:
code="GR"

In [4]:
response = session.get(f'https://api.openaq.org/v2/locations?limit=1000&page=1&offset=0&sort=desc&radius=1000&country_id={code}&order_by=lastUpdated&dumpRaw=false')

In [5]:
test = response.json()

In [6]:
# test["results"]

In [7]:
def visualize_humidity_on_map(data_dict):
    # Initialize lists to store data for multiple locations
    latitudes = []
    longitudes = []
    humidity_values = []
    display_names = []

    # Loop through the results and extract relevant data for each location
    for result in data_dict.get('results', []):
        measurements = result.get('parameters', [])
        for measurement in measurements:
            if measurement['parameter'] == 'humidity':
                humidity_value = measurement['lastValue']
                display_name = measurement['displayName']
                latitude = result['coordinates']['latitude']
                longitude = result['coordinates']['longitude']

                latitudes.append(latitude)
                longitudes.append(longitude)
                humidity_values.append(humidity_value)
                display_names.append(display_name)

    if not latitudes or not longitudes or not humidity_values:
        print("Humidity data not found.")
        return

    # Create the visualization
    fig = go.Figure()

    # Add custom HTML markers as map markers
    custom_markers = [
        f"""<span style='font-size: 20px; color: blue;'>&#x1F4A7;</span> Humidity: {humidity}%<br>{display_name}"""
        for humidity, display_name in zip(humidity_values, display_names)
    ]

    # Add a single scatter mapbox trace with all locations
    fig.add_trace(go.Scattermapbox(
        lat=latitudes,
        lon=longitudes,
        mode='markers+text',
        marker=go.scattermapbox.Marker(
            size=20,
            color=humidity_values,
            colorscale='Viridis',  # You can choose other color scales as well
            colorbar=dict(title='Humidity (%)')
        ),
        text=custom_markers,
        hoverinfo='text'
    ))
    
    # # Add a scatter mapbox trace for each location
    # for lat, lon, humidity_value in zip(latitudes, longitudes, humidity_values):
    #     fig.add_trace(go.Scattermapbox(
    #         lat=[lat],
    #         lon=[lon],
    #         mode='markers',
    #         marker=go.scattermapbox.Marker(
    #             size=20,
    #             color=humidity_value,
    #             colorscale='Viridis',  # You can choose other color scales as well
    #             colorbar=dict(title='Humidity (%)')
    #         ),
    #         text=[f'Humidity: {humidity_value}%'],
    #         hoverinfo='text'
    #     ))

    # Update map layout
    fig.update_layout(
        mapbox=dict(
            style='open-street-map',  # Choose the desired map style
            zoom=5,  # Adjust the initial zoom level as needed
            center=dict(lat=sum(latitudes) / len(latitudes), lon=sum(longitudes) / len(longitudes))
        ),
        margin=dict(l=0, r=0, t=0, b=0)
    )

    # Show the map
    fig.show()

visualize_humidity_on_map(test)


In [9]:
countries = response.json()["results"]

In [11]:
data_dict = {item["name"]: item for item in countries}


In [13]:
data_dict["Greece"]

{'id': 128,
 'code': 'GR',
 'name': 'Greece',
 'datetimeFirst': '2019-05-31T20:50:02+00:00',
 'datetimeLast': '2023-07-23T13:49:17+00:00',
 'parameters': [{'id': 1,
   'name': 'pm10',
   'units': 'µg/m³',
   'displayName': None},
  {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': None},
  {'id': 3, 'name': 'o3', 'units': 'µg/m³', 'displayName': None},
  {'id': 4, 'name': 'co', 'units': 'µg/m³', 'displayName': None},
  {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'displayName': None},
  {'id': 6, 'name': 'so2', 'units': 'µg/m³', 'displayName': None},
  {'id': 19, 'name': 'pm1', 'units': 'µg/m³', 'displayName': None},
  {'id': 95, 'name': 'pressure', 'units': 'hpa', 'displayName': None},
  {'id': 100, 'name': 'temperature', 'units': 'c', 'displayName': None},
  {'id': 125, 'name': 'um003', 'units': 'particles/cm³', 'displayName': None},
  {'id': 126, 'name': 'um010', 'units': 'particles/cm³', 'displayName': None},
  {'id': 128, 'name': 'temperature', 'units': 'f', 'displayName': 

In [10]:
def filter_by_name(data, target_name):
    filtered_data = [item for item in data if item["name"] == target_name]
    return filtered_data

target_name = "Greece"
filtered_countries = filter_by_name(countries, target_name)
filtered_countries

[{'id': 128,
  'code': 'GR',
  'name': 'Greece',
  'datetimeFirst': '2019-05-31T20:50:02+00:00',
  'datetimeLast': '2023-07-23T13:49:17+00:00',
  'parameters': [{'id': 1,
    'name': 'pm10',
    'units': 'µg/m³',
    'displayName': None},
   {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': None},
   {'id': 3, 'name': 'o3', 'units': 'µg/m³', 'displayName': None},
   {'id': 4, 'name': 'co', 'units': 'µg/m³', 'displayName': None},
   {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'displayName': None},
   {'id': 6, 'name': 'so2', 'units': 'µg/m³', 'displayName': None},
   {'id': 19, 'name': 'pm1', 'units': 'µg/m³', 'displayName': None},
   {'id': 95, 'name': 'pressure', 'units': 'hpa', 'displayName': None},
   {'id': 100, 'name': 'temperature', 'units': 'c', 'displayName': None},
   {'id': 125, 'name': 'um003', 'units': 'particles/cm³', 'displayName': None},
   {'id': 126, 'name': 'um010', 'units': 'particles/cm³', 'displayName': None},
   {'id': 128, 'name': 'temperature', 'units': 